In [ ]:
!apt-get update
!apt-get install portaudio19-dev python3-pyaudio # Removed the extra space before this line

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,160 kB]
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:14 http://archive.ubuntu.com

In [ ]:
!pip install pyaudio --no-cache-dir  # Reinstall pyaudio, bypassing cache

In [ ]:
!pip install SpeechRecognition

In [ ]:
import speech_recognition as sr

recognizer = sr.Recognizer()

def audio_to_text(audio_file):
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "Unrecognizable speech"

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)


In [ ]:
context_mapping = {'urgent': 1, 'not urgent': 0}
context_encoded = [context_mapping[context] for context in contexts]

In [ ]:
import re
import speech_recognition as sr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Speech Recognition
recognizer = sr.Recognizer()

def audio_to_text(audio_file):
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            return "Unrecognizable speech"

# Text Preprocessing
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# Example dataset
transcriptions = ["Call 911, I need help", "I fell down, please help", "Remind me to take meds", "Call my daughter please"]
contexts = ["urgent", "urgent", "not urgent", "not urgent"]
intent_labels = ["Emergency Services", "Contact Caregiver", "Set Reminder", "Contact Family"]

# Clean transcriptions
cleaned_transcriptions = [clean_text(t) for t in transcriptions]

# Convert text to features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_transcriptions)

# Encode context and intent
context_mapping = {'urgent': 1, 'not urgent': 0}
context_encoded = [context_mapping[c] for c in contexts]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(intent_labels)

# X contains TF-IDF features, context_encoded contains context information, y contains encoded intent labels.


In [ ]:
import os
import librosa
import numpy as np

def load_and_preprocess_audio_files(dataset_path, sr=16000, n_mfcc=13):
    # List all subfolders (commands or categories) in the dataset path
    categories = os.listdir(dataset_path)

    data = []
    labels = []

    for idx, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)

        for filename in os.listdir(category_path):
            if filename.endswith(".mp3"):
                # Full path to the audio file
                file_path = os.path.join(category_path, filename)

                # Load the audio file with librosa, and resample to 16kHz
                audio, sample_rate = librosa.load(file_path, sr=sr)

                # Preprocess the audio data: extract MFCC features
                mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)

                # Take the mean of the MFCC coefficients along the time axis
                mfccs_mean = np.mean(mfccs.T, axis=0)

                # Append to data list
                data.append(mfccs_mean)
                labels.append(idx)  # Label by category index

    # Convert to numpy arrays
    return np.array(data), np.array(labels)

# Set the path to your dataset
dataset_path = "/content/voicedataset"

# Load and preprocess audio files
audio_features, labels = load_and_preprocess_audio_files(dataset_path)



In [ ]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Function to process speech from an audio file
def recognize_from_audiofile(file_path):
    try:
        # Open the audio file
        with sr.AudioFile(file_path) as source:
            print(f"Processing file: {file_path}")
            audio = recognizer.record(source)  # Read the entire file into an audio object

            # Recognize speech using Google Web Speech API
            print("Recognizing...")
            command = recognizer.recognize_google(audio)
            print(f"Detected command: {command}")
            return command
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print(f"Request error from Google Speech Recognition service; {e}")
    except FileNotFoundError:
        print(f"Audio file not found: {file_path}")
    return None

if __name__ == "__main__":
    # Example audio file path
    audio_file_path = "/content/WhatsApp Audio 2024-10-15 at 12.06.29_bc1c411a.waptt (online-audio-converter.com).wav"

    # Call the function to recognize speech from the audio file
    command = recognize_from_audiofile(audio_file_path)
    if command:
        # Process command (e.g., analyze intent, issue emergency alert)
        pass



Processing file: /content/WhatsApp Audio 2024-10-15 at 12.06.29_bc1c411a.waptt (online-audio-converter.com).wav
Recognizing...
Detected command: hi hello my my name is Nikita hi hello we just completed lab
